In [233]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [234]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [235]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [236]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [237]:
train_features = train.drop(['SalePrice','Id'],axis=1)
test = test.drop(['Id'],axis=1)

In [238]:
train_features.shape

(1460, 79)

In [239]:
test.shape

(1459, 79)

# Deal with missing values

In [240]:
def DropMissingValue(data:pd,threshold = 0.3) -> pd: # drop missing value that exceed threshold of number of observations
    df_null = data.isnull().sum()
    null_index = df_null[df_null >= threshold*data.shape[0]].index
    data = data.drop(null_index,axis=1)
    return data

In [241]:
train_new = DropMissingValue(train_features)
test_new = DropMissingValue(test)

In [242]:
train_new.columns == test_new.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

# Fill nan with mean for float numbers except for year,fill nan with mode for categorical values

In [243]:
train_null = train_new.isnull().sum().sort_values(ascending=False) != 0
train_null_idx = train_null[train_null].index

In [244]:
test_null = test_new.isnull().sum().sort_values(ascending=False) != 0
test_null_idx = test_null[test_null].index

In [245]:
train_new[train_null_idx].dtypes

LotFrontage     float64
GarageFinish     object
GarageType       object
GarageCond       object
GarageQual       object
GarageYrBlt     float64
BsmtExposure     object
BsmtFinType2     object
BsmtFinType1     object
BsmtCond         object
BsmtQual         object
MasVnrArea      float64
MasVnrType       object
Electrical       object
dtype: object

In [246]:
train_null_idx

Index(['LotFrontage', 'GarageFinish', 'GarageType', 'GarageCond', 'GarageQual',
       'GarageYrBlt', 'BsmtExposure', 'BsmtFinType2', 'BsmtFinType1',
       'BsmtCond', 'BsmtQual', 'MasVnrArea', 'MasVnrType', 'Electrical'],
      dtype='object')

In [247]:
train_new[['LotFrontage','MasVnrArea']] = train_new[['LotFrontage','MasVnrArea']].fillna(train_new.mean().iloc[0])

In [248]:
train_new[[ 'GarageCond', 'GarageType', 'GarageYrBlt','GarageFinish', 'GarageQual', 'BsmtFinType2', 'BsmtExposure',
           'BsmtFinType1', 'BsmtCond', 'BsmtQual', 'MasVnrType','Electrical']] = train_new[[ 'GarageCond', 'GarageType', 'GarageYrBlt','GarageFinish', 'GarageQual', 'BsmtFinType2', 'BsmtExposure',
           'BsmtFinType1', 'BsmtCond', 'BsmtQual', 'MasVnrType','Electrical']].fillna(train_new.mode().iloc[0])

In [249]:
test_null_idx

Index(['LotFrontage', 'GarageFinish', 'GarageCond', 'GarageQual',
       'GarageYrBlt', 'GarageType', 'BsmtCond', 'BsmtQual', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'MasVnrType', 'MasVnrArea', 'MSZoning',
       'Functional', 'BsmtHalfBath', 'BsmtFullBath', 'Utilities',
       'Exterior2nd', 'Exterior1st', 'KitchenQual', 'TotalBsmtSF',
       'GarageCars', 'SaleType', 'BsmtUnfSF', 'GarageArea', 'BsmtFinSF2',
       'BsmtFinSF1'],
      dtype='object')

In [250]:
test_new[['LotFrontage','MasVnrArea','BsmtHalfBath','BsmtFullBath','BsmtUnfSF','BsmtFinSF2','BsmtFinSF1','TotalBsmtSF',
          'GarageArea']] = test_new[['LotFrontage','MasVnrArea','BsmtHalfBath','BsmtFullBath','BsmtUnfSF','BsmtFinSF2','BsmtFinSF1','TotalBsmtSF',
          'GarageArea']].fillna(train_new.mean().iloc[0])

In [251]:
test_new[[ 'GarageCond', 'GarageQual', 'GarageYrBlt',
       'GarageFinish', 'GarageType', 'BsmtCond', 'BsmtQual', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'MasVnrType', 'MSZoning', 'Utilities', 'Functional', 'BsmtUnfSF',
       'SaleType', 'Exterior2nd', 'Exterior1st', 'GarageCars', 'KitchenQual']] = test_new[[ 'GarageCond', 'GarageQual', 'GarageYrBlt',
       'GarageFinish', 'GarageType', 'BsmtCond', 'BsmtQual', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'MasVnrType', 'MSZoning', 'Utilities', 'Functional', 'BsmtUnfSF',
       'SaleType', 'Exterior2nd', 'Exterior1st', 'GarageCars', 'KitchenQual']].fillna(train_new.mode().iloc[0])

# Categorical features

In [252]:
df = pd.concat([train_new,test_new])  #Combine train and test data

In [253]:
def Dummies(data:pd) -> pd:  # dummy all categorical features
    object_features = data.dtypes[df.dtypes == object].index
    dummies = pd.get_dummies(data[object_features])
    df_new = pd.concat([data,dummies],axis=1)
    df_new.drop(columns=object_features,inplace=True)
    df_new = df_new.loc[:,~df_new.columns.duplicated()]
    return df_new

In [254]:
df_new = Dummies(df)

In [255]:
df_new.shape

(2919, 270)

# Split the train and test back

In [256]:
train_new = df_new.iloc[:1460,:]
test_new = df_new.iloc[1460:,:]

# Do some feature engineering

In [257]:
train_new.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,...,0,0,0,1,0,0,0,0,1,0


In [258]:
np.corrcoef(train_new['OverallQual'],train_new['OverallCond'])  
# these two features looks correlated as names implies, but they are not correlated much

array([[ 1.        , -0.09193234],
       [-0.09193234,  1.        ]])

YearBuilt: Original construction date<br>
YearRemodAdd: Remodel date (same as construction date if no remodeling or additions)<br>
MoSold: Month Sold (MM)<br>
YrSold: Year Sold (YYYY)


House's life and remodel  may be a critical feature to price the house

In [259]:
# do something with YearBuilt and YearRemodAdd and MoSold: Month Sold (MM) and YrSold: Year Sold (YYYY)
train_new['LifeBuilt'] = train_new['MoSold']/12 + train_new['YrSold'] - train_new['YearBuilt']
train_new['LifeRemod'] = train_new['MoSold']/12 + train_new['YrSold'] - train_new['YearRemodAdd']

In [260]:
test_new['LifeBuilt'] = test_new['MoSold']/12 + test_new['YrSold'] - test_new['YearBuilt']
test_new['LifeRemod'] = test_new['MoSold']/12 + test_new['YrSold'] - test_new['YearRemodAdd']

In [261]:
train_new = train_new.drop(['MoSold','YrSold','YearBuilt','YearRemodAdd'],axis=1)
test_new = test_new.drop(['MoSold','YrSold','YearBuilt','YearRemodAdd'],axis=1)

YearBuilt and YearRemodAdd : Result does not improve

Next: <br>
Try to do something with square feet : <br>

BsmtFinSF2: Type 2 finished square feet<br>
BsmtUnfSF: Unfinished square feet of basement area<br>
TotalBsmtSF: Total square feet of basement area<br>

1stFlrSF: First Floor square feet<br>
2ndFlrSF: Second floor square feet<br>
LowQualFinSF: Low quality finished square feet (all floors)<br>
GrLivArea: Above grade (ground) living area square feet<br>

GarageArea: Size of garage in square feet<br>

WoodDeckSF: Wood deck area in square feet<br>

OpenPorchSF: Open porch area in square feet<br>

EnclosedPorch: Enclosed porch area in square feet<br>
3SsnPorch: Three season porch area in square feet<br>
ScreenPorch: Screen porch area in square feet<br>
PoolArea: Pool area in square feet<br>

# Build model

In [262]:
X = train_new
y = train['SalePrice']

In [263]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import xgboost as xgb

In [264]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [265]:
xgb_model = xgb.XGBRegressor()

In [266]:
xgb_model.fit(X_train,y_train)

[02:30:25] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

# HyperParameter Optimization

In [295]:
from sklearn.model_selection import RandomizedSearchCV

In [296]:
n_estimator = [20,30,50,100,120,200,300,500,900,1000]
booster = ['gbtree','gblinear']
base_score = [0.25,0.5,0.75,1]
max_depth = [2,3,5,10,15]
learning_rate = [0.05,0.1,0.15,0.2,0.25,0.3]
min_childweight = [1,2,3,4,5]

In [297]:
hyperparametergrid = {'n_estimator':n_estimator,
                      'max_depth':max_depth,
                     'learning_rate':learning_rate,
                     'min_child_weight':min_childweight,
                     'booster':booster,
                     'base_score':base_score}

In [298]:
random_cv = RandomizedSearchCV(estimator=xgb_model,
                              param_distributions=hyperparametergrid,
                              cv=5,scoring='neg_mean_squared_error',return_train_score=True,verbose=5,random_state=123)

In [299]:
random_cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimator=50, min_child_weight=4, max_depth=3, learning_rate=0.15, booster=gbtree, base_score=0.5 
[02:32:53] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimator=50, min_child_weight=4, max_depth=3, learning_rate=0.15, booster=gbtree, base_score=0.5, score=(train=-128459015.428, test=-1170361378.702), total=   0.7s
[CV] n_estimator=50, min_child_weight=4, max_depth=3, learning_rate=0.15, booster=gbtree, base_score=0.5 
[02:32:54] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV]  n_estimator=50, min_child_weight=4, max_depth=3, learning_rate=0.15, booster=gbtree, base_score=0.5, score=(train=-129776339.007, test=-907883835.594), total=   0.7s
[CV] n_estimator=50, min_child_weight=4, max_depth=3, learning_rate=0.15, booster=gbtree, base_score=0.5 
[02:32:54] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

[CV]  n_estimator=50, min_child_weight=4, max_depth=3, learning_rate=0.15, booster=gbtree, base_score=0.5, score=(train=-136682547.560, test=-1339125295.701), total=   0.7s
[CV] n_estimator=300, min_child_weight=5, max_depth=5, learning_rate=0.25, booster=gblinear, base_score=1 
[02:32:57] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  n_estimator=300, min_child_weight=5, max_depth=5, learning_rate=0.25, booster=gblinear, base_score=1, score=(train=-1005766963.033, test=-3484098506.062), total=   0.5s
[CV] n_estimator=300, min_child_weight=5, max_depth=5, learning_rate=0.25, booster=gblinear, base_score=1 
[02:32:57] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  n_estimator=300, min_child_weight=5, max_depth=5, learning_rate=0.25, booster=gblinear, base_score=1, sc

[CV]  n_estimator=100, min_child_weight=2, max_depth=2, learning_rate=0.1, booster=gblinear, base_score=0.75, score=(train=-1388211438.080, test=-1102761101.007), total=   0.5s
[CV] n_estimator=100, min_child_weight=2, max_depth=2, learning_rate=0.1, booster=gblinear, base_score=0.75 
[02:33:07] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  n_estimator=100, min_child_weight=2, max_depth=2, learning_rate=0.1, booster=gblinear, base_score=0.75, score=(train=-1274349640.355, test=-1655207426.960), total=   0.5s
[CV] n_estimator=200, min_child_weight=4, max_depth=2, learning_rate=0.2, booster=gbtree, base_score=1 
[02:33:07] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  n_estimator=200, min_child_weight=4, max_depth=2, learning_rate=0.2, booster=gbtree, base_score=1, 

[CV]  n_estimator=300, min_child_weight=5, max_depth=3, learning_rate=0.05, booster=gbtree, base_score=0.25, score=(train=-319475127.710, test=-643382335.151), total=   0.7s
[CV] n_estimator=300, min_child_weight=5, max_depth=3, learning_rate=0.05, booster=gbtree, base_score=0.25 
[02:33:19] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  n_estimator=300, min_child_weight=5, max_depth=3, learning_rate=0.05, booster=gbtree, base_score=0.25, score=(train=-294095010.713, test=-801584418.078), total=   0.7s
[CV] n_estimator=300, min_child_weight=5, max_depth=3, learning_rate=0.05, booster=gbtree, base_score=0.25 
[02:33:20] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  n_estimator=300, min_child_weight=5, max_depth=3, learning_rate=0.05, booster=gbtree, base_score=0.25,

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   30.4s finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=None,
                                          objective='reg:linear',
                                          random_st...
                   param_distributions={'base_score': [0.25, 0.5, 0.75, 1],
                                        'booster': ['gbtree', 'gblinear'],
                                      

In [300]:
random_cv.best_estimator_

XGBRegressor(base_score=0.75, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.3, max_delta_step=0,
             max_depth=5, min_child_weight=4, missing=None, n_estimator=50,
             n_estimators=100, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

# Optimize model using hyperparameters

In [301]:
xgb_model_opt = xgb.XGBRegressor(base_score=0.75, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.3, max_delta_step=0,
             max_depth=5, min_child_weight=4, missing=None, n_estimator=50,
             n_estimators=100, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [302]:
xgb_model_opt.fit(X_train,y_train)

[02:33:42] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.75, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.3, max_delta_step=0,
             max_depth=5, min_child_weight=4, missing=None, n_estimator=50,
             n_estimators=100, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

# Predict using Optimized model

In [303]:
preds = xgb_model_opt.predict(X_test)

In [304]:
RMSE = sqrt(mean_squared_error(y_test,preds))

In [305]:
print(RMSE)  # error is higher than yesterday after doing the features of YearBuilt,YearRemodAdd

29557.569676226074


# Predict Test dataset

In [306]:
sub = pd.read_csv('sample_submission.csv')

In [307]:
sub.head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [308]:
saleprice = xgb_model_opt.predict(test_new )

In [309]:
sub['SalePrice'] = saleprice

In [310]:
sub.to_csv('submission.csv',index=False)

In [311]:
sub.head()

,Id,SalePrice
0,1461,129907.054688
1,1462,153005.515625
2,1463,179946.734375
3,1464,194769.203125
4,1465,202055.171875
